In [ ]:
import pandas as pd

In [ ]:
true = pd.read_csv('/content/drive/MyDrive/True.csv')
fake = pd.read_csv('/content/drive/MyDrive/Fake.csv')

In [ ]:
true['label'] = 1
fake['label'] = 0

In [ ]:
frames = [true.loc[:][:5000], fake.loc[:][:5000]]
df = pd.concat(frames)

In [ ]:
with open('/content/drive/MyDrive/Combined/Combined.txt', 'a') as f:
    f.write(df.to_string(index=False))

In [ ]:
X = df['text']
y = df['label']
corpus = df['text'].values
X

,text
0,WASHINGTON (Reuters) - The head of a conservat...
1,WASHINGTON (Reuters) - Transgender people will...
2,WASHINGTON (Reuters) - The special counsel inv...
3,WASHINGTON (Reuters) - Trump campaign adviser ...
4,SEATTLE/WASHINGTON (Reuters) - President Donal...
...,...
4995,It s no secret Republicans are salivating to f...
4996,Republicans are about to lose a huge source of...
4997,A pawn working for Donald Trump claimed that w...
4998,Fox News is desperate to sabotage Hillary Clin...


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1")

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [ ]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [ ]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
250/250 [==============================] - 120s 471ms/step - loss: 0.4507 - accuracy: 0.8429 - precision: 0.8306 - recall: 0.8607
Epoch 2/5
250/250 [==============================] - 120s 481ms/step - loss: 0.2407 - accuracy: 0.9369 - precision: 0.9302 - recall: 0.9444
Epoch 3/5
250/250 [==============================] - 118s 473ms/step - loss: 0.1800 - accuracy: 0.9515 - precision: 0.9434 - recall: 0.9604
Epoch 4/5
250/250 [==============================] - 119s 476ms/step - loss: 0.1515 - accuracy: 0.9546 - precision: 0.9492 - recall: 0.9604
Epoch 5/5
250/250 [==============================] - 117s 469ms/step - loss: 0.1343 - accuracy: 0.9576 - precision: 0.9502 - recall: 0.9657


In [ ]:
model.evaluate(X_test, y_test)
#Accuracy: 96.85%
#Precision: 97.49%
#Recall: 96.23%
#F1 - SCORE: 96.86%

63/63 [==============================] - 30s 475ms/step - loss: 0.1108 - accuracy: 0.9685 - precision: 0.9749 - recall: 0.9623


[0.11077118664979935,
 0.968500018119812,
 0.9748743772506714,
 0.9623016119003296]